In [39]:
from __future__ import annotations
from copy import deepcopy

class Dice():
    data: list = [0, 1, 2, 3, 4, 5]
    TIP_MAPPINGS = {"S": [(3, 0), (0, 1), (1, 2), (2, 3)], "W": [(5, 0), (0, 4), (4, 2), (2, 5)]}
    TIP_MAPPINGS["E"] = [(i[1], i[0]) for i in TIP_MAPPINGS["W"]]
    TIP_MAPPINGS["N"] = [(i[1], i[0]) for i in TIP_MAPPINGS["S"]]
    step: int
    location: tuple  # In the format of y, x
    LOCATION_MAPPINGS = {"N": (-1, 0), "E": (0, 1), "S": (1, 0), "W": (0, -1)}
    score: int
    DIRECTION_TO_DATA_MAPPINGS = {"S": 3, "W": 5, "N": 1, "E": 4}
    # top: int
    # down: int
    # front: int
    # back: int
    # left: int
    # right: int
    # adjacencies: dict[str, list[int]]

    def __init__(self, data = None, location = (5, 0), step = 0, score = 0):
        # self.top = self.data[0]
        # self.down = self.data[2]
        # self.front = self.data[1]
        # self.back = self.data[3]
        # self.left = self.data[4]
        # self.right = self.data[5]
        if data != None:
            self.data = data
        self.location = location
        self.step = step
        self.score = score
    
    def tip(self, direction: str) -> None:
        """Tip/roll the cube in one of the cardinal directions. Currently supporting N and E for a Naive recursion."""
        temp = None
        for a, b in self.TIP_MAPPINGS[direction]:  # shifts data by one, wraps around.
            if temp != None:
                temp2 = self.data[b]
                self.data[b] = temp
                temp = temp2
            else:
                temp = self.data[b]
                self.data[b] = self.data[a]
        loc_transform = self.LOCATION_MAPPINGS[direction]
        # print(direction, "old loc" , self.location, end = "; ")
        self.location = (self.location[0] + loc_transform[0], self.location[1] + loc_transform[1])
        # print(direction, "new loc" , self.location)
        self.step += 1

    def new_face(self, datum) -> None:
        """Replaces current upwards facing face's value."""
        self.data[0] = datum

    def copy(self) -> Dice:
        """Create new copy of the Dice."""
        new = Dice(deepcopy(self.data), self.location, self.step, self.score)
        return new

    def inc_score(self) -> None:
        self.score += self.data[0] * self.step

    def get_face_value(self, direction) -> float:
        """"""
        return self.data[self.DIRECTION_TO_DATA_MAPPINGS[direction]]

    def __str__(self) -> None:
        """Prints a surface unwrap of a dice cube."""
        a = f"""
        __|{self.data[3]}|__
        |{self.data[4]}|{self.data[0]}|{self.data[5]}|
        __|{self.data[1]}|__
        __|{self.data[2]}|__
        {self.location} {self.step}
        """
        return a


class Grid():

    grid = [
    [57, 33, 132, 268, 492, 732],
    [81, 123, 240, 443, 353, 508],
    [186, 42, 195, 704, 452, 228],
    [-7, 2, 357, 452, 317, 395],
    [5, 23, -4, 592, 445, 620],
    [0, 77, 32, 403, 337, 452]
    ]
    path = []

    def __init__(self, goal = (0, 5), start = (5, 0), grid = None):
        """"""
        if grid is None:
            pass
        else:
            self.grid = grid
        self.goal = goal
        self.start = start

    def simulate(self):
        """"""
        start_die = Dice([0, 0, 0, 0, 0, 0], self.start)
        path = self.RecursiveAdvance([], start_die)
        print("mypath", path)
        print("catched path", self.path)

    def RecursiveAdvance(self, inp_path: list, inp_cube: Dice) -> list:
        """"""
        childs = []
        # print("cube loc", inp_cube.location, "step", inp_cube.step, "scorechange", inp_cube.step*inp_cube.data[0], "score", inp_cube.score, "id", id(inp_cube))
        if inp_cube.location == self.goal:
            self.path = inp_path
            print("HORYA", inp_path)
            return inp_path
        
        for direction in ["N", "E", "S", "W"]:
            print(inp_cube.location, "|", inp_cube.location[0] + inp_cube.LOCATION_MAPPINGS[direction][0],
                inp_cube.location[1] + inp_cube.LOCATION_MAPPINGS[direction][1])
            cube = inp_cube.copy()
            path = deepcopy(inp_path)

            try:
                grid_score = self.grid\
                    [cube.location[0]+cube.LOCATION_MAPPINGS[direction][0]]\
                    [cube.location[1]+cube.LOCATION_MAPPINGS[direction][1]]
            except IndexError:
                continue

            if cube.location[0]+cube.LOCATION_MAPPINGS[direction][0] < 0 or \
                cube.location[1]+cube.LOCATION_MAPPINGS[direction][1] < 0:
                continue

            if cube.get_face_value(direction) == 0:
                cube.tip(direction)
                cube.new_face((grid_score-cube.score)/cube.step)

            elif cube.get_face_value(direction) != 0:
                if ((grid_score - cube.score) / (cube.step + 1)) == cube.get_face_value(direction):
                    print("grid score", grid_score, "expected face", (grid_score - cube.score) / (cube.step + 1), "actual face", cube.get_face_value(direction))
                    print("Good Match")
                    cube.tip(direction)
                    print(cube)
                    
                else:
                    print("expected face", (grid_score - cube.score) / (cube.step + 1), "actual face", cube.get_face_value(direction))
                    continue

            cube.inc_score()
            path.append(grid_score)
            childs.append((path, cube))
        # return [self.RecursiveAdvance(childs[index][0], childs[index][1]) for index in range(len(childs))]
        if len(childs) == 0:
            return []
        else:
            # print("CHILD", childs)
            return max([self.RecursiveAdvance(childs[index][0], childs[index][1]) for index in range(len(childs))])
        

Test Cases

In [40]:
# # 3x3 grid.
# test1_raw = [
#     [5, 31, 67],
#     [6, 10, 5],
#     [0, -7, 3]
# ]
# test1_grid = Grid((0, 2), (2, 0), test1_raw)
# test1_grid.simulate()

In [41]:
# # Full sized grid possible with only N and E rolls.
# test2_raw = [
#     [57, 33, 132, 268, 492, 159],
#     [81, 123, 240, 443, 353, 119],
#     [186, 42, 195, 704, 452, 110],
#     [-7, 2, 357, 452, 317, 94],
#     [5, 23, -4, 592, 445, 73],
#     [0, 1, 11, 20, 44, 49]
#     ]
# test2_grid = Grid(grid = test2_raw)
# test2_grid.simulate()

In [42]:
# Full sized grid possible with only N and E rolls alt.
test2_raw = [
    [49, 73, 94, 143, 161, 221],
    [44, 123, 240, 103, 353, 119],
    [20, 42, 195, 96, 452, 110],
    [11, 2, 357, 60, 317, 94],
    [2, 13, 25, 45, 445, 73],
    [0, 1, 15, 20, 44, 49]
    ]
test2_grid = Grid(grid = test2_raw)
test2_grid.simulate()

(5, 0) | 4 0
(5, 0) | 5 1
(5, 0) | 6 0
(5, 0) | 5 -1
(4, 0) | 3 0
(4, 0) | 4 1
(4, 0) | 5 0
(4, 0) | 4 -1
(3, 0) | 2 0
expected face 3.0 actual face 2.0
(3, 0) | 3 1
(3, 0) | 4 0
expected face -3.0 actual face 2.0
(3, 0) | 3 -1
(3, 1) | 2 1
expected face 10.0 actual face 2.0
(3, 1) | 3 2
expected face 88.75 actual face 4.5
(3, 1) | 4 1
expected face 2.75 actual face 2.0
(3, 1) | 3 0
expected face 2.25 actual face 4.5
(4, 1) | 3 1
(4, 1) | 4 2
expected face 4.0 actual face 2.0
(4, 1) | 5 1
(4, 1) | 4 0
expected face -3.6666666666666665 actual face 2.0
(3, 1) | 2 1
expected face 10.0 actual face 5.5
(3, 1) | 3 2
expected face 88.75 actual face 2.0
(3, 1) | 4 1
expected face 2.75 actual face 5.5
(3, 1) | 3 0
expected face 2.25 actual face 2.0
(5, 1) | 4 1
expected face 3.0 actual face 5.5
(5, 1) | 5 2
expected face 3.5 actual face 2.0
(5, 1) | 6 1
(5, 1) | 5 0
expected face -0.25 actual face 2.0
(5, 0) | 4 0
expected face 0.6666666666666666 actual face 2.0
(5, 0) | 5 1
(5, 0) | 6 0
(5, 0)

In [43]:
# # Full sized grid possible with only N and E rolls, piror to fixing role direction..
# test2_raw = [
#     [57, 33, 132, 268, 152, 212],
#     [81, 123, 240, 108, 116, 119],
#     [186, 42, 50, 80, 452, 110],
#     [9, 27, 35, 452, 317, 94],
#     [1, 23, -4, 592, 445, 73],
#     [0, 1, 11, 20, 44, 49]
#     ]
# test2_grid = Grid(grid = test2_raw)
# test2_grid.simulate()

(5, 0) | 4 0
(5, 0) | 5 1
(5, 0) | 6 0
(5, 0) | 5 -1
(4, 0) | 3 0
(4, 0) | 4 1
(4, 0) | 5 0
(4, 0) | 4 -1
(3, 0) | 2 0
expected face 59.0 actual face 1.0
(3, 0) | 3 1
(3, 0) | 4 0
expected face -2.6666666666666665 actual face 1.0
(3, 0) | 3 -1
(3, 1) | 2 1
expected face 3.75 actual face 1.0
(3, 1) | 3 2
expected face 2.0 actual face 4.0
(3, 1) | 4 1
expected face -1.0 actual face 1.0
(3, 1) | 3 0
expected face -4.5 actual face 4.0
(4, 1) | 3 1
(4, 1) | 4 2
expected face -9.0 actual face 1.0
(4, 1) | 5 1
(4, 1) | 4 0
expected face -7.333333333333333 actual face 1.0
(3, 1) | 2 1
expected face 3.75 actual face 11.0
(3, 1) | 3 2
expected face 2.0 actual face 1.0
(3, 1) | 4 1
expected face -1.0 actual face 11.0
(3, 1) | 3 0
expected face -4.5 actual face 1.0
(5, 1) | 4 1
expected face 5.5 actual face 11.0
(5, 1) | 5 2
expected face 2.5 actual face 1.0
(5, 1) | 6 1
(5, 1) | 5 0
expected face -0.25 actual face 1.0
(5, 0) | 4 0
expected face 0.3333333333333333 actual face 1.0
(5, 0) | 5 1
(5, 

Real Case

In [44]:
#Real Case
myGrid = Grid()
myGrid.simulate()
# starting_die = Dice([0, 0, 0, 0, 0, 0])
# path = myGrid.RecursiveAdvance([], starting_die)
# print("mypath", path)
# print("catched path", myGrid.path)

(5, 0) | 4 0
(5, 0) | 5 1
(5, 0) | 6 0
(5, 0) | 5 -1
(4, 0) | 3 0
(4, 0) | 4 1
(4, 0) | 5 0
(4, 0) | 4 -1
(3, 0) | 2 0
expected face 64.33333333333333 actual face 5.0
(3, 0) | 3 1
(3, 0) | 4 0
expected face 4.0 actual face 5.0
(3, 0) | 3 -1
(3, 1) | 2 1
expected face 10.0 actual face 5.0
(3, 1) | 3 2
expected face 88.75 actual face -6.0
(3, 1) | 4 1
expected face 5.25 actual face 5.0
(3, 1) | 3 0
expected face -2.25 actual face -6.0
(4, 1) | 3 1
(4, 1) | 4 2
expected face -9.0 actual face 5.0
(4, 1) | 5 1
(4, 1) | 4 0
expected face -6.0 actual face 5.0
(3, 1) | 2 1
expected face 10.0 actual face 9.0
(3, 1) | 3 2
expected face 88.75 actual face 5.0
(3, 1) | 4 1
expected face 5.25 actual face 9.0
(3, 1) | 3 0
expected face -2.25 actual face 5.0
(5, 1) | 4 1
expected face -13.5 actual face 9.0
(5, 1) | 5 2
expected face -11.25 actual face 5.0
(5, 1) | 6 1
(5, 1) | 5 0
expected face -19.25 actual face 5.0
(5, 0) | 4 0
expected face 1.6666666666666667 actual face 5.0
(5, 0) | 5 1
(5, 0) | 6